In [1]:
import numpy as np
import torch
import os
import sys
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.insert(0, parent_dir)
from model import AttentionModel
from model_PCA_correlation import AttentionModel_PCA
from dcascore import *
from utils import read_fasta_alignment, remove_duplicate_sequences, add_PCA_coords

# back to original path (in PLM)
sys.path.pop(0)  # Removes the parent_dir from sys.path
from model import AttentionModel

from plm_gen_methods import generate_plm_n_save, generate_coords_n_save, generate_multiple_targets_n_save
from seq_utils import read_tensor_from_txt, set_seed, letters_to_nums, modify_seq 


In [2]:
print(current_dir)
print(parent_dir)

/Users/marzioformica/Desktop/EPFL/Master/StageLBS/PCA_gen_AI/CODE/AttentionDCA_python/src/PLM
/Users/marzioformica/Desktop/EPFL/Master/StageLBS/PCA_gen_AI/CODE/AttentionDCA_python/src


In [2]:
"""
    Load Q, K, V matrices from jdoms (after training)
"""
set_seed()
H = 64
d= 10
N = 174
n_epochs = 500
nb_PCA_comp=2
loss_type = 'without_J'
family = 'jdoms' #'jdoms_bacteria_train2'
cwd = parent_dir
Q_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_PCA_brute_force_35_bins/Q_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
K_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_PCA_brute_force_35_bins/K_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
V_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_PCA_brute_force_35_bins/V_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
H,d,N=Q_1.shape
q=V_1.shape[1]


In [3]:
model=AttentionModel(H,d,N,q,Q=Q_1,V=V_1,K=K_1)
torch.sum(model.Q-Q_1)
device = Q_1.device
L = Q_1.shape[-1]
W=attention_heads_from_model(model,Q_1,K_1,V_1)
print(W.shape)

i_indices = torch.arange(L, device=device).unsqueeze(1)
j_indices = torch.arange(L, device=device).unsqueeze(0)
mask = (i_indices != j_indices).float().unsqueeze(0)  # shape (1, L, L)
W = W * mask
    
# Compute Jtens
Jtens = torch.einsum('hri,hab->abri', W, V_1)  # Shape: (q, q, L, L)
q = Jtens.shape[0]
N = Jtens.shape[2]
print(q)
print(N)
print(Jtens.shape)

torch.Size([64, 65, 65])
35
65
torch.Size([35, 35, 65, 65])


In [ ]:
"""
    Generate sequences with PLM random initialization
"""

##############################################################


In [ ]:

save_dir = "generated_sequences_brute_force"
N_seqs = 30000
save_name = "generated_sequences_randinit_30000"
#generate_plm_n_save(save_dir, save_name, Jtens, N_seqs=N_seqs, init_sequence=None,nb_PCA_comp=2,PCA_comp_list=np.array([20,20]))
N_seqs=15000
betas = [0.01, 0.1, 0.5, 1, 2, 4, 10]

for b in betas:
    save_name = f"gen_seqs_randinit_Ns{N_seqs}_b{b}_b_PCA_1_PCA_comp_24_20"
    generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=None,nb_PCA_comp=nb_PCA_comp, beta=b,PCA_comp_list=np.array([24,20]))
    if b!=1:
        save_name = f"gen_seqs_randinit_Ns{N_seqs}_b_1_b_PCA{b}_PCA_comp_24_20"
        generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=None, beta=1,nb_PCA_comp=nb_PCA_comp,beta_PCA=b,PCA_comp_list=np.array([24,20]))

"""
    Generate sequences with PLM initialization from a sequence
"""
file_test_data=r"C:\Users\youss\OneDrive\Bureau\master epfl\MA2\TP4 De los Rios\git_test\PLM-gen-DCA\Attention-DCA-main\CODE\DataAttentionDCA\jdoms\jdoms_bacteria_train2.fasta"
seq_data_test=read_fasta_alignment(file_test_data,0.8)

print(seq_data_test.shape)
seq_data_test_filtered,_=remove_duplicate_sequences(seq_data_test)
seq_data_test_filtered=seq_data_test_filtered.T
seq_data_test_filtered_with_PCA= add_PCA_coords(seq_data_test_filtered,35) #####DONT FORGET TO CHANGE NUMBER OF BINS IF NECESSARY


#init_sequence = 'DYYQVLGVPKDADAKSIKKAFRKLARKYHPDVNPGDKEAERKFKEANEANEVLSDPEKRKKYD'
init_sequence_num = seq_data_test_filtered_with_PCA[0]
print(init_sequence_num[-2:])
ratio = 0.4
init_sequence_num = modify_seq(init_sequence_num, ratio,nb_PCA_comp=2)
N_seqs=30000
save_name = f"gen_seqs_w_init_seq_Ns{N_seqs}_r{ratio}"

#generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=init_sequence_num,nb_PCA_comp=nb_PCA_comp)
N_seqs=15000
betas = [0.01, 0.1, 0.5, 1, 2, 4, 10]
for b in betas:
    save_name = f"gen_seqs_w_init_seq_Ns{N_seqs}_r{ratio}_b{b}"
    generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=init_sequence_num.copy(),nb_PCA_comp=nb_PCA_comp, beta=b)
    if b!=1:
        save_name = f"gen_seqs_w_init_seq_Ns{N_seqs}_r{ratio}_b_1_b_PCA{b}"
        generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=init_sequence_num.copy(), beta=1,nb_PCA_comp=nb_PCA_comp,beta_PCA=b)

100%|██████████| 15000/15000 [07:40<00:00, 32.58it/s]


Generated sequences (letters): ['PTEARANCWRDKGDVSETWGWIVFQYSYRWAFH-PEWQYVCWPSIDIQTCTQGWTPFTYRKRV', 'PTEARANCWRDKGDVSETWGWIVFQYSYRWAFH-PEWQSVCWPSIDIQTCTQGWTPFTYRKRV', 'PTEARANCWRDKGDVSETWGWIQFQYSYRWAFH-PEWQSVCWPSIDIQTCTQGWTPFTYRKRV', 'PTEARANCWRDKGDESETWGWIQFQYSYRWAFH-PEWQSVCWPSIDIQTCTQGWTPFTYRKRV', 'PTEARANCWRDTGDESETWGWIQFQYSYRWAFH-PEWQSVCWPSIDIQTCTQGWTPFTYRKRV']
Generated sequences saved to generated_sequences_brute_forcePCA_comp


100%|██████████| 15000/15000 [07:29<00:00, 33.34it/s]


Generated sequences (letters): ['VDMSFKLNTWLEFLIIVHCGFHQGTHRMVVWQHGKH-YQVLGAMKNGLMDNDFI-DCLSYR-A', 'VDMSFKLKTWLEFLIIVHCGFHQGTHRMVVWQHGKH-YQVLGAMKNGLMDNDFI-DCLSYR-A', 'VDMSFKLKTWLEFLIIVHCKFHQGTHRMVVWQHGKH-YQVLGAMKNGLMDNDFI-DCLSYR-A', 'VDMSFKLKTWLEFLIIVHCKFHQGTHRMVVWQHGKH-YQVLGAMKRGLMDNDFI-DCLSYR-A', 'VDMSFKLKTWLEFLIIVHCKFHQGTHRMVVWQHGKH-YQVLGAMKRGLFDNDFI-DCLSYR-A']
Generated sequences saved to generated_sequences_brute_forcePCA_comp


100%|██████████| 15000/15000 [07:34<00:00, 32.98it/s]


...

Generated sequences (letters): ['DYYEILGVSKNASKDEIKKAYRKLAKKYHPDVN-GDKEAEKKFKEINEAYEVLSDEEKRKKYD', 'DYYEILGVSKNASKDEIKKAYRKLAKKYHPDVN-GDKEAEKKFKEINEAYEVLSDEEKRKKYD', 'DYYEILGVSKNASKDEIKKAYRKLAKKYHPDVN-GDKEAEKKFKEINEAYEVLSDEEKRKKYD', 'DYYEILGVSKNASKDEIKKAYRKLAKKYHPDVN-GDKEAEKKFKEINEAYEVLSDEEKRKKYD', 'DYYEILKVSKNASKDEIKKAYRKLAKKYHPDVN-GDKEAEKKFKEINEAYEVLSDEEKRKKYD']
Generated sequences saved to generated_sequences_brute_forcePCA_comp


100%|██████████| 15000/15000 [07:35<00:00, 32.91it/s]


Generated sequences (letters): ['DPYEVLGVPYEAKAEEVKEANRKLSFVCRPDNNPGDRFAKERMIRVTEAYEE-----------', 'DPYEVLGVPYEAKAEEVKEANRKLSFVCRPDNNPGDRFAKERMIRVTEAYEE-----------', 'DPYEVLGVPYEAKAEEIKEANRKLSFVCRPDNNPGDRFAKERMIRVTEAYEE-----------', 'DPYEVLGVPYEAKAEEIKEANRKLSFVCRPDNNPGDRFAKERMIRITEAYEE-----------', 'DPYEVLGVPYEAKAEEIKEANRKLSFVCRPDNNPGDRFAKERMIRITEAYEE-----------']
Generated sequences saved to generated_sequences_brute_forcePCA_comp


100%|██████████| 15000/15000 [07:35<00:00, 32.94it/s]


Generated sequences (letters): ['DPYKILGVSKDASDEEIKKAYRKLAKKYHPDLNPGDKEAEEKFKEINAAYELIKK--------', 'DPYKILGVSKDASDEEIKKAYRKLAKKYHPDLNPGDKEAEEKFKEINAAYELIKK--------', 'DPYKILGVSKDASDEEIKKAYRKLAKKYHPDLNPGDKEAEEKFKEINAAYELIKK--------', 'DPYKILGVSKDASDEEIKKAYRKLAKKYHPDLNPGDKEAEEKFKEINAAYELIKK--------', 'DPYKILGVSKDASDEEIKKAYRKLAKKYHPDLNPGDKEAEEKFKEINAAYELIKK--------']
Generated sequences saved to generated_sequences_brute_forcePCA_comp


Brute force specific generation

In [6]:
save_dir = "generated_sequences_brute_force_specific"
N_seqs = 30000
save_name = "generated_sequences_randinit_30000"
#generate_plm_n_save(save_dir, save_name, Jtens, N_seqs=N_seqs, init_sequence=None,nb_PCA_comp=2,PCA_comp_list=np.array([20,20]))
N_seqs=15000
betas = [10,20,50]
targey_list=[[24,20], [27,5],[8,8]]
for target in targey_list:
    for b in betas:
        save_name = f"gen_seqs_randinit_Ns{N_seqs}_b{b}_b_PCA_1_PCA_comp_{target[0]}_{target[1]}"
        #generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=None,nb_PCA_comp=nb_PCA_comp, beta=b,PCA_comp_list=np.array([24,20]))
        if b!=1:
            save_name = f"gen_seqs_randinit_Ns{N_seqs}_b_2_b_PCA{b}_PCA_comp_{target[0]}_{target[1]}"
            generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=None, beta=2,nb_PCA_comp=nb_PCA_comp,beta_PCA=b,PCA_comp_list=np.array(target))


  0%|          | 0/15000 [00:00<?, ?it/s]

100%|██████████| 15000/15000 [07:59<00:00, 31.25it/s]


Generated sequences (letters): ['PWHFTQV-PLL-CFNGDVVDTTHFQWNHC-QWYFWMFRMAYRVITGVNIWRRMWHTSRYFAAG', 'PWHFTQV-PLL-CFNGDVVDTTHSQWNHC-QWYFWMFRMAYRVITGVNIWRRMWHTSRYFAAG', 'PWHFTQV-PLL-CFNGDVVDTTHSQWNHC-QWYFWMFRMARRVITGVNIWRRMWHTSRYFAAG', 'PWHFTQV-PLL-CFNGDVVDTTHSQWNHC-QWYFWMFRMARRVITGVNIWRRMWHPSRYFAAG', 'PWHFTQV-PLL-C-NGDVVDTTHSQWNHC-QWYFWMFRMARRVITGVNIWRRMWHPSRYFAAG']
Generated sequences saved to generated_sequences_brute_force_specificPCA_comp


100%|██████████| 15000/15000 [07:59<00:00, 31.31it/s]


Generated sequences (letters): ['CPVDWIGENIYWNVWHPQWHAPSCYGKPLTCTFW-FKVT-M-GMDT-PYKKKRAFCMRSLKED', 'CPVDWIGENIYWNVWHPQWHAPSCYGKPLTCTTW-FKVT-M-GMDT-PYKKKRAFCMRSLKED', 'CPYDWIGENIYWNVWHPQWHAPSCYGKPLTCTTW-FKVT-M-GMDT-PYKKKRAFCMRSLKED', 'CPYDWIGENIYWNVWHPQWHAPSCYGKPLTCTTW-FKVT-M-GMDT-PYKKKRAFCMRSLKED', 'CPYDWIGENKYWNVWHPQWHAPSCYGKPLTCTTW-FKVT-M-GMDT-PYKKKRAFCMRSLKED']
Generated sequences saved to generated_sequences_brute_force_specificPCA_comp


...

Generated sequences (letters): ['IYIFWYLLVILNDEKGFQKTIGLRDTSYNPSVSLVMAKMT-NNPYQFKCCILIQTSPVESPLH', 'IYIFWYLLVILVDEKGFQKTIGLRDTSYNPSVSLVMAKMT-NNPYQFKCCILIQTSPVESPLH', 'IYIFWLLLVILVDEKGFQKTIGLRDTSYNPSVSLVMAKMT-NNPYQFKCCILIQTSPVESPLH', 'IYIFWLLLVILVDEKGFQKTIGLRDTSYNPSVSLVMAKMT-NNPYQFKCCILIQTSPVESPLH', 'IYIFWLLLVILVDEKGFQKTIGLRDTSYNPSVSLVMAKMT-NNPYQFKACILIQTSPVESPLH']
Generated sequences saved to generated_sequences_brute_force_specificPCA_comp


100%|██████████| 15000/15000 [07:57<00:00, 31.41it/s]


Generated sequences (letters): ['VKYPDRSNER-CWENY-NAFFWRYP-AVIPSNQLTDGGGAHNRTFFTLDRFATDPNYFMNYNV', 'VKYPDRSNER-CWENY-NAFFWRYP-AVIPSNQLTDGGGAHNRTFFTLDRFPTDPNYFMNYNV', 'VKYPDRSNER-CWENY-NAFFWRYP-AVIPSNQLTDGGGAHNRTFFTLDRFPTDPNYFMNYNV', 'VKYPDRSNER-CWENY-NAFFWRYP-AVIPSNQLTDGFGAHNRTFFTLDRFPTDPNYFMNYNV', 'VKYPDRSNER-CWENY-NAFFWRYP-AVIPSNQLTDGFGAHNRTFFTLSRFPTDPNYFMNYNV']
Generated sequences saved to generated_sequences_brute_force_specificPCA_comp


100%|██████████| 15000/15000 [07:56<00:00, 31.46it/s]


Generated sequences (letters): ['-FQ-ETKLMSD-DNHWCLSMYSFFGHRWDWVTV-YDGTSMRINIGAKSQMHEEFYEHTIM-WW', '-FQ-ETKLMSD-DNHWCLSMYSFFGHRWEWVTV-YDGTSMRINIGAKSQMHEEFYEHTIM-WW', '-FQ-ETKLMSD-DNHWCLSMYRFFGHRWEWVTV-YDGTSMRINIGAKSQMHEEFYEHTIM-WW', '-FQ-ETKLMSE-DNHWCLSMYRFFGHRWEWVTV-YDGTSMRINIGAKSQMHEEFYEHTIM-WW', '-FQ-ETKLMSE-TNHWCLSMYRFFGHRWEWVTV-YDGTSMRINIGAKSQMHEEFYEHTIM-WW']
Generated sequences saved to generated_sequences_brute_force_specificPCA_comp


## MODEL 2

In [2]:
H = 64
d= 10
N = 174
n_epochs = 500
nb_PCA_comp=2
loss_type = 'without_J'
family = 'jdoms' #'jdoms_bacteria_train2'
cwd = parent_dir
Q_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_youss/Q_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
K_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_youss/K_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
V_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_youss/V_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
H,d,N=Q_1.shape
q=V_1.shape[1]

In [3]:
model=AttentionModel(H,d,N,q,Q=Q_1,V=V_1,K=K_1)
torch.sum(model.Q-Q_1)
device = Q_1.device
L = Q_1.shape[-1]
W=attention_heads_from_model(model,Q_1,K_1,V_1)
print(W.shape)

i_indices = torch.arange(L, device=device).unsqueeze(1)
j_indices = torch.arange(L, device=device).unsqueeze(0)
mask = (i_indices != j_indices).float().unsqueeze(0)  # shape (1, L, L)
W = W * mask
    
# Compute Jtens
Jtens = torch.einsum('hri,hab->abri', W, V_1)  # Shape: (q, q, L, L)
q = Jtens.shape[0]
N = Jtens.shape[2]
print(q)
print(N)
print(Jtens.shape)
print(Jtens.shape[-1])

torch.Size([64, 63, 63])
21
63
torch.Size([21, 21, 63, 63])
63


In [4]:
Q_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_PCA_2models_35_bins/Q_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
K_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_PCA_2models_35_bins/K_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
V_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_PCA_2models_35_bins/V_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
H,d,N1=Q_1.shape
_,_,N2=K_1.shape
_,q1,q2=V_1.shape
model=AttentionModel_PCA(H,d,N1,N2,q1,q2,Q=Q_1,V=V_1,K=K_1)
torch.sum(model.Q-Q_1)
device = Q_1.device
L = Q_1.shape[-1]
W=attention_heads_from_model(model,Q_1,K_1,V_1)
print(W.shape)

# i_indices = torch.arange(L, device=device).unsqueeze(1)
# j_indices = torch.arange(L, device=device).unsqueeze(0)
# mask = (i_indices != j_indices).float().unsqueeze(0)  # shape (1, L, L)
# W = W * mask
    
# Compute Jtens
Jtens_PCA = torch.einsum('hri,hab->abri', W, V_1)  # Shape: (q, q, L, L)
q = Jtens.shape[0]
N = Jtens.shape[2]
print(q)
print(N)
print(Jtens_PCA.shape)

torch.Size([64, 63, 2])
21
63
torch.Size([21, 35, 63, 2])


In [9]:
"""
    Generate sequences with PLM random initialization
"""
save_dir = "generated_sequences_2Model_PCA"
N_seqs = 30000
save_name = "generated_sequences_randinit_30000"
#generate_plm_n_save(save_dir, save_name, Jtens, N_seqs=N_seqs, init_sequence=None,nb_PCA_comp=2,PCA_comp_list=np.array([20,20]),J_PCA=Jtens_PCA)
N_seqs=15000
betas = [0.01, 0.1, 0.5, 1, 2, 4, 10]

for b in betas:
    save_name = f"gen_seqs_randinit_Ns{N_seqs}_b{b}_b_PCA_1"
    #generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=None,nb_PCA_comp=nb_PCA_comp, beta=b,J_PCA=Jtens_PCA,PCA_comp_list=np.array([23,23]))
    if b!=1:
        save_name = f"gen_seqs_randinit_Ns{N_seqs}_b_1_b_PCA{b}"
        generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=None, beta=1,nb_PCA_comp=nb_PCA_comp,J_PCA=Jtens_PCA,beta_PCA=b,PCA_comp_list=np.array([23,23]))

##############################################################

"""
    Generate sequences with PLM initialization from a sequence
"""
file_test_data=r"C:\Users\youss\OneDrive\Bureau\master epfl\MA2\TP4 De los Rios\git_test\PLM-gen-DCA\Attention-DCA-main\CODE\DataAttentionDCA\jdoms\jdoms_bacteria_train2.fasta"
seq_data_test=read_fasta_alignment(file_test_data,0.8)

print(seq_data_test.shape)
seq_data_test_filtered,_=remove_duplicate_sequences(seq_data_test)
seq_data_test_filtered=seq_data_test_filtered.T
seq_data_test_filtered_with_PCA= add_PCA_coords(seq_data_test_filtered,35) #####DONT FORGET TO CHANGE NUMBER OF BINS IF NECESSARY


#init_sequence = 'DYYQVLGVPKDADAKSIKKAFRKLARKYHPDVNPGDKEAERKFKEANEANEVLSDPEKRKKYD'
init_sequence_num = seq_data_test_filtered_with_PCA[0]
print(init_sequence_num[-2:])
ratio = 0.4
init_sequence_num = modify_seq(init_sequence_num, ratio,nb_PCA_comp=2)
N_seqs=30000
save_name = f"gen_seqs_w_init_seq_Ns{N_seqs}_r{ratio}"

#generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=init_sequence_num,nb_PCA_comp=nb_PCA_comp,J_PCA=Jtens_PCA)

"""
    Generate sequences with PLM initialization from a sequence different betas
"""
# init_sequence = 'DYYQVLGVPKDADAKSIKKAFRKLARKYHPDVNPGDKEAERKFKEANEANEVLSDPEKRKKYD'
# init_sequence_num = letters_to_nums(init_sequence)
# ratio = 0.1
# init_sequence_num = modify_seq(init_sequence_num, ratio)

N_seqs=15000
betas = [0.01, 0.1, 0.5, 1, 2, 4,10]

for b in betas:
    save_name = f"gen_seqs_w_init_seq_Ns{N_seqs}_r{ratio}_b{b}"
    #generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=init_sequence_num,nb_PCA_comp=nb_PCA_comp, beta=b,J_PCA=Jtens_PCA)
    if b!=1:
        save_name = f"gen_seqs_w_init_seq_Ns{N_seqs}_r{ratio}_b_1_b_PCA{b}"
        generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=init_sequence_num, beta=1,nb_PCA_comp=nb_PCA_comp,J_PCA=Jtens_PCA,beta_PCA=b)

100%|██████████| 15000/15000 [07:29<00:00, 33.40it/s]


Generated sequences (letters): ['PSAEEILYWFHPCHIRVGQKL-YTYGSSAWEVYYYRIACLAM-ENWDAAFGHK-VSFPMCCIL', 'PSAEEILYWFHPCHIRVGQKL-YTYGSSAWEVYYYRIACLAM-ENWDAAFGHK-VEFPMCCIL', 'PSAEEILYWFHPCHIRVGAKL-YTYGSSAWEVYYYRIACLAM-ENWDAAFGHK-VEFPMCCIL', 'PSAEEILYWFHPCHPRVGAKL-YTYGSSAWEVYYYRIACLAM-ENWDAAFGHK-VEFPMCCIL', 'PSAEEILYWFHPLHPRVGAKL-YTYGSSAWEVYYYRIACLAM-ENWDAAFGHK-VEFPMCCIL']
Generated sequences saved to generated_sequences_2Model_PCAPCA_comp


100%|██████████| 15000/15000 [07:39<00:00, 32.62it/s]


Generated sequences (letters): ['HSIPWGWKSYEWETFEWTFELDWSGPM-VNSEDYTYFRHWNFWMPAFWGSETPG-RCGIKWLM', 'HSIPWGWKSYEWETFEWKFELDWSGPM-VNSEDYTYFRHWNFWMPAFWGSETPG-RCGIKWLM', 'HSIPWGWDSYEWETFEWKFELDWSGPM-VNSEDYTYFRHWNFWMPAFWGSETPG-RCGIKWLM', 'HSIPWGWDSYEWETFEWKFELDWSGPM-VNSEDYTYFRHWNFWMPAFWGPETPG-RCGIKWLM', 'HSIPWGWDSYEWETFEWKFELDWSGPM-VNSEYYTYFRHWNFWMPAFWGPETPG-RCGIKWLM']
Generated sequences saved to generated_sequences_2Model_PCAPCA_comp


100%|██████████| 15000/15000 [07:35<00:00, 32.90it/s]


...

Generated sequences (letters): ['----ILGLDARADAKDIRKAYRLLAKEYHPDTG-------ERFAAITEAYELL----------', '----ILGLDARADAKDIRKAYRLLAKEYHPDTG-------ERFAAITEAYELL----------', '----ILGLDARADAKDIRKAYRLLAKEYHPDTG-------ERFAAITEAYELL----------', '----ILGLDARADAKDIRKAYRLLAKEYHPDTG-------ERFAAITEAYELL----------', '----ILGLDARADAKDIRKAYRLLAKEYHPDTG-------ERFAAITEAYELL----------']
Generated sequences saved to generated_sequences_2Model_PCAPCA_comp


100%|██████████| 15000/15000 [07:13<00:00, 34.58it/s]


Generated sequences (letters): ['------------------------------D--------------------------------', '------------------------------D--------------------------------', '------------------------------D--------------------------------', '------------------------------D--------------------------------', '------------------------------D--------------------------------']
Generated sequences saved to generated_sequences_2Model_PCAPCA_comp


100%|██████████| 15000/15000 [07:13<00:00, 34.63it/s]


Generated sequences (letters): ['------------------------------D--------------------------------', '------------------------------D--------------------------------', '------------------------------D--------------------------------', '------------------------------D--------------------------------', '------------------------------D--------------------------------']
Generated sequences saved to generated_sequences_2Model_PCAPCA_comp


In [12]:
init_sequence_num = seq_data_test_filtered_with_PCA[0]
print(init_sequence_num[-2:])
ratio = 0.4
init_sequence_num = modify_seq(init_sequence_num, ratio,nb_PCA_comp=2)
print(init_sequence_num)

[17  5]
[13 20 12  5 17  9 20  7  8  3 13  0 15 13  2  3  7  8  8  0 19 14  3 11
  7  6  7 11  6 12  2 13 19  5  2 11 12  9  0  3  3  8 14 14  3  9 11  1
 12  1  2 19  9 15  8 16 15 20 20 11 20 20 20 17  5]


In [6]:
save_dir = "generated_sequences_2Model_specific"
N_seqs = 30000
save_name = "generated_sequences_randinit_30000"
#generate_plm_n_save(save_dir, save_name, Jtens, N_seqs=N_seqs, init_sequence=None,nb_PCA_comp=2,PCA_comp_list=np.array([20,20]))
N_seqs=15000
betas = [6,10,15,25]
targey_list=[[24,20], [27,5],[8,8]]
for target in targey_list:
    for b in betas:
        save_name = f"gen_seqs_randinit_Ns{N_seqs}_b{b}_b_PCA_1_PCA_comp_{target[0]}_{target[1]}"
        #generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=None,nb_PCA_comp=nb_PCA_comp, beta=b,PCA_comp_list=np.array([24,20]))
        if b!=1:
            save_name = f"gen_seqs_randinit_Ns{N_seqs}_b_1_b_PCA{b}_PCA_comp_{target[0]}_{target[1]}"
            generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=None, beta=1,nb_PCA_comp=nb_PCA_comp,beta_PCA=b,PCA_comp_list=np.array(target),J_PCA=Jtens_PCA)


100%|██████████| 15000/15000 [11:50<00:00, 21.12it/s]


Generated sequences (letters): ['FKWCETEGCEICCIFQPWANYDQFTTMNEKYRA--ADNGDIPNYCTWWRYSGCMDKYSIPECT', 'FKWCETEGCEICCIFQPWANYDQFLTMNEKYRA--ADNGDIPNYCTWWRYSGCMDKYSIPECT', 'FKWCETEGCEICCIFQPWANYDQFLTMNEKYRA--ADNGDIPNYCTWWRYSLCMDKYSIPECT', 'FKWCETEGCEICCIFQPWANYDQFLTMNEKYRA--ADNGDIPLYCTWWRYSLCMDKYSIPECT', 'FKWCETEGCEICCIFQPWANYDQFLTMNEKYRA--ADNGDIPLYC-WWRYSLCMDKYSIPECT']
Generated sequences saved to generated_sequences_2Model_specificPCA_comp


100%|██████████| 15000/15000 [11:15<00:00, 22.19it/s]


Generated sequences (letters): ['CYVQKQVTIPFKWRVGHEQ-ITN-QCGETRA-NPCASQSTMDT-GPGMCHWDRLPASHMKINI', 'CYVQKQVTIPFKWRVGHEQ-ITN-QCGETPA-NPCASQSTMDT-GPGMCHWDRLPASHMKINI', 'CYVQKQVTI-FKWRVGHEQ-ITN-QCGETPA-NPCASQSTMDT-GPGMCHWDRLPASHMKINI', 'CYVQKQVTI-EKWRVGHEQ-ITN-QCGETPA-NPCASQSTMDT-GPGMCHWDRLPASHMKINI', 'CYVQKQVTI-EKWRVGHEQ-ITN-QCGETPA-NPCASQSTMDT-GPGECHWDRLPASHMKINI']
Generated sequences saved to generated_sequences_2Model_specificPCA_comp


100%|██████████| 15000/15000 [08:42<00:00, 28.72it/s]


...

Generated sequences (letters): ['ISWPLICGVMWFVNITQKPAVPCPILEDMLEKHSEDKCWHDRQWWYGNI-AYDAC-LIRATA-', 'ISWPLICGVMWFVNITQKPAVACPILEDMLEKHSEDKCWHDRQWWYGNI-AYDAC-LIRATA-', 'ISWPLICGVMWFVNITQKPAVACPILEDMLEKHSEDKCWHDRQWWYANI-AYDAC-LIRATA-', 'ISWPLICGVMWFANITQKPAVACPILEDMLEKHSEDKCWHDRQWWYANI-AYDAC-LIRATA-', 'ISWPLICGVMWFANATQKPAVACPILEDMLEKHSEDKCWHDRQWWYANI-AYDAC-LIRATA-']
Generated sequences saved to generated_sequences_2Model_specificPCA_comp


100%|██████████| 15000/15000 [07:40<00:00, 32.59it/s]


Generated sequences (letters): ['EIDHIFTDEYDINT-GEGKKGIDSVPKRKWPKGEAVFQAMRQ-DWA-ISVDP-FYNCCG-DQS', 'EIAHIFTDEYDINT-GEGKKGIDSVPKRKWPKGEAVFQAMRQ-DWA-ISVDP-FYNCCG-DQS', 'EIAHIFTDEYDINT-GEGKKGIDSVPKRKWPKGEAVFQAMRQ-DWA-ISVAP-FYNCCG-DQS', 'EIAAIFTDEYDINT-GEGKKGIDSVPKRKWPKGEAVFQAMRQ-DWA-ISVAP-FYNCCG-DQS', 'EIAAIFTDEYDIAT-GEGKKGIDSVPKRKWPKGEAVFQAMRQ-DWA-ISVAP-FYNCCG-DQS']
Generated sequences saved to generated_sequences_2Model_specificPCA_comp


100%|██████████| 15000/15000 [07:40<00:00, 32.60it/s]


Generated sequences (letters): ['EWRERFCTADVARDWIMANWQMIKQYLVWPPLNMANWYNLINMNEWFFLLAGSRSSPQQKSKL', 'EWRERFCTADVARDWIMANWQMIKQYLVWPPLNMANWYNLINMNEWFFLLAGSRSSPQQASKL', 'EWRERFCTADVARDWIMANWQMIKQYLVWPPLNMANWYNLINMNEWFFLLAGSRSSPQQASKL', 'EWRERFCTADVARDWIMANWQMIKQYLVWPPLNMANWYNLINMNEWFFLLAGSRSSPAQASKL', 'EWRERFCTADVARDWIMANWQMIKQYLVWPPLNMANWYNLANMNEWFFLLAGSRSSPAQASKL']
Generated sequences saved to generated_sequences_2Model_specificPCA_comp


## ADDED - NEW MODEL

In [10]:
"""
    Generate sequences with PLM random initialization
"""
save_dir = "generated_sequences_2Model_PCA"
N_seqs = 15000
save_name = "generated_sequences_randinit_30000_25_6"
PCA_comp_list=np.array([25,6])
generate_plm_n_save(save_dir, save_name, Jtens, N_seqs=N_seqs, init_sequence=None,nb_PCA_comp=2,PCA_comp_list=PCA_comp_list,J_PCA=Jtens_PCA)

print("done generating sequences with PLM initialization from a random sequence")


100%|██████████| 15000/15000 [03:19<00:00, 75.22it/s]


Generated sequences (letters): ['N-K-MSKIYDWSKRATYRAVTMLMQFCSGPHTECDIHWDFNNKNESADQLC-NHKT-VCHQLF', '--K-MSKIYDWSKRATYRAVTMLMQFCSGPHTECDIHWDFNNKNESADQLC-NHKT-VCHQLF', '--K-MSKIYDWSKRATYRAVTMLMQFCSGPHTECDI-WDFNNKNESADQLC-NHKT-VCHQLF', '--K-MSKIYDWSKRATYRAVTALMQFCSGPHTECDI-WDFNNKNESADQLC-NHKT-VCHQLF', '--K-MSKIYDWS-RATYRAVTALMQFCSGPHTECDI-WDFNNKNESADQLC-NHKT-VCHQLF']
Generated sequences saved to generated_sequences_2Model_PCAPCA_comp
done generating sequences with PLM initialization from a random sequence


In [ ]:
##############################################################

"""
    Generate sequences with PLM initialization from a sequence
"""
import platform

if platform.system() == "Darwin":  # macOS
    base_path = '/Users/marzioformica/Desktop/EPFL/Master/StageLBS/PCA_gen_AI/CODE/DataAttentionDCA/jdoms'
else:  # assume Windows for Youss
    base_path = r"C:\Users\youss\OneDrive\Bureau\master epfl\MA2\TP4 De los Rios\git_test\PLM-gen-DCA\Attention-DCA-main\CODE\DataAttentionDCA\jdoms"

file_test_data = f"{base_path}/jdoms_bacteria_train2.fasta"
seq_data_test=read_fasta_alignment(file_test_data,0.8)

print(seq_data_test.shape)
seq_data_test_filtered,_=remove_duplicate_sequences(seq_data_test)
seq_data_test_filtered=seq_data_test_filtered.T
seq_data_test_filtered_with_PCA= add_PCA_coords(seq_data_test_filtered,35) #####DONT FORGET TO CHANGE NUMBER OF BINS IF NECESSARY


#init_sequence = 'DYYQVLGVPKDADAKSIKKAFRKLARKYHPDVNPGDKEAERKFKEANEANEVLSDPEKRKKYD'
init_sequence_num = seq_data_test_filtered_with_PCA[0]
print(init_sequence_num[-2:])
print("initial sequence number:", init_sequence_num)
ratio = 0.4
init_sequence_num = modify_seq(init_sequence_num, ratio,nb_PCA_comp=2)
N_seqs=15000
save_name = f"gen_seqs_w_init_seq_Ns{N_seqs}_r{ratio}_20_20"

generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=init_sequence_num,nb_PCA_comp=nb_PCA_comp,PCA_comp_list=PCA_comp_list, J_PCA=Jtens_PCA)
print("done generating sequences with PLM initialization from an initial sequence sequence")


Total sequences read: 14502
Sequences after filtering: 14502
(63, 14502)
Removing duplicate sequences...
Done: 14502 -> 14502 sequences after removing duplicates.
[17  5]
initial sequence number: [11 12 19  3 17  9  5  7  8  3  5  0 15 13  2  3  7  8  8  0 19 14  3  9
  0  8  8 19  6 12  2 13 19  5  2 11 12  9  0  3  3  8 10 14  3  9 11  3
  0 19  2 19  9 15  8 11 15 20 20 20 20 20 20 17  5]


100%|██████████| 15000/15000 [03:31<00:00, 70.87it/s]


Generated sequences (letters): ['NWYEVVGIWETASSYEIKKAY-ELAKKYGWDQYVDNPLCEEKIIDLRHFYDYRSKAS--HG--', 'NWYEVVGIWETASSYEIKKAY-ELAKKYGWDQYVDNPLCEEKIIDLRHFYDYRSKAS--HG--', 'NWYEVVGIWETASSYEIKKAY-ELAKKYGWDQYVDNPLCEEKIIDIRHFYDYRSKAS--HG--', 'NWYEVVGIWETASSYEIKKAY-ELAKKYGWDQYVDNPLCEEKIIDIRHFYDYRSKAS--HG--', '-WYEVVGIWETASSYEIKKAY-ELAKKYGWDQYVDNPLCEEKIIDIRHFYDYRSKAS--HG--']
Generated sequences saved to generated_sequences_2Model_PCAPCA_comp
done generating sequences with PLM initialization from an initial sequence sequence


## Generate from multiple coords

In [7]:
save_dir = "generated_sequences_2Model_PCA_multiple"
N_seqs = 10000
save_name_prefix = f'generated_sequences_randinit_{N_seqs}_mult'
# take 10 random coordinates from the PCA space (0-34)
target_coords_list = [(np.random.randint(0, 35), np.random.randint(0, 35)) for _ in range(10)]
generate_multiple_targets_n_save(save_dir, save_name_prefix, J=Jtens, target_coords_list=target_coords_list, N_seqs=N_seqs, init_sequence=None, beta=1, nb_PCA_comp=0, J_PCA=Jtens_PCA, beta_PCA=1)

Mismatch of PCA tensor and nb PCA components indicated
number of PCA components doesn't match size of PCA list


100%|██████████| 10000/10000 [03:38<00:00, 45.82it/s]


[0] Generated sequences for target (17, 15) (first 5): ['NSNMCQTRGYQVGNQLVGTVMGGALQIWPADGCERWQEM-SYTG-IIT--M-NPVVCSTGTDM', 'NSNMCQTRGYQVGNALVGTVMGGALQIWPADGCERWQEM-SYTG-IIT--M-NPVVCSTGTDM', 'NSNMCQTRGYQVGNALVGTVMGGALQIWPADGCERWQEM-SYTG-IIT-YM-NPVVCSTGTDM', 'NSN-CQTRGYQVGNALVGTVMGGALQIWPADGCERWQEM-SYTG-IIT-YM-NPVVCSTGTDM', 'NSN-CQTRGYQVGNVLVGTVMGGALQIWPADGCERWQEM-SYTG-IIT-YM-NPVVCSTGTDM']
Mismatch of PCA tensor and nb PCA components indicated
number of PCA components doesn't match size of PCA list


100%|██████████| 10000/10000 [03:41<00:00, 45.09it/s]


[1] Generated sequences for target (20, 34) (first 5): ['MILWMSLGEMHKPFIHV-TA-SRYCAPDIMIRHDWRMDPALGPEPMGFDHCQVDC-IAGIYNA', 'MILWMSLGEMHKPFIHV-TA-SRYCAPDIMIRHDWRMDPALGPEPMGFDHCQVDCLIAGIYNA', 'MILWMSLGEMHKPFIHVDTA-SRYCAPDIMIRHDWRMDPALGPEPMGFDHCQVDCLIAGIYNA', 'MILWMSLGEMHKPFIHVDTA-SRYCAPDIMIRHDWRMDPALGPEPMGFDHCRVDCLIAGIYNA', 'MILWMSLGEMHKPFIHVDTA-SRYCAPDIMIRHDWRMDPALGPERMGFDHCRVDCLIAGIYNA']
Mismatch of PCA tensor and nb PCA components indicated
number of PCA components doesn't match size of PCA list


...

[7] Generated sequences for target (31, 27) (first 5): ['FHKYVAQQRAHTRASWRCLDRTGMLEAA-GSLPEFSHISVPIFEQWYIYPVMYPNFG-SGVKY', 'FHKYVAQQRAHTRASWRCLDRTGMLEAA-GSLPEFSHISVPIFEQWYIYPVMHPNFG-SGVKY', 'FHKYVAQQRAHTRASWRCLDRSGMLEAA-GSLPEFSHISVPIFEQWYIYPVMHPNFG-SGVKY', 'FHKYVAQQRAHTRASWRCLDRKGMLEAA-GSLPEFSHISVPIFEQWYIYPVMHPNFG-SGVKY', 'FHKYVAQQRAHTRASWRCLDRKGMLEAA-GSLPEFRHISVPIFEQWYIYPVMHPNFG-SGVKY']
Mismatch of PCA tensor and nb PCA components indicated
number of PCA components doesn't match size of PCA list


100%|██████████| 10000/10000 [04:14<00:00, 39.25it/s]


[8] Generated sequences for target (33, 21) (first 5): ['YPMNFILFQDSWDMKTYGVNKQMLRREQRAHSTQIWAWMVPKMIAQLY--TEPHEQGEC-SNS', 'YPMNFILKQDSWDMKTYGVNKQMLRREQRAHSTQIWAWMVPKMIAQLY--TEPHEQGEC-SNS', 'YPMNFILKQDSWDMKTYGVNKQMLRREQRAHSTQIWAWMVPKMIAQLY--TEPHEQYEC-SNS', 'YPMNFILKQDSWDMKTYGVNKQMLRREQRAHSTQIWAWMVPKMIAQLY--TEPHEQYEA-SNS', 'YPMNFILKQDSWDMKTYGVNKQMLRREQRAHSTQIWAWMVPKMIAQLY-VTEPHEQYEA-SNS']
Mismatch of PCA tensor and nb PCA components indicated
number of PCA components doesn't match size of PCA list


100%|██████████| 10000/10000 [04:08<00:00, 40.27it/s]


[9] Generated sequences for target (1, 1) (first 5): ['MIAHF-YFDIPTTTFRYGKPAWIEETILFGNDMPARWLI-PDHWC-WEYRYVQSIHAWYQRFI', 'MIAHF-YFDYPTTTFRYGKPAWIEETILFGNDMPARWLI-PDHWC-WEYRYVQSIHAWYQRFI', 'MIAHF-YFDYPTTTFRYGKPAWIEETILFGNDMPARWLI-PDHWC-WEYRYVQSIHAWYQRFI', 'MIAHF-YFDYPTTTFRYGKPAWGEETILFGNDMPARWLI-PDHWC-WEYRYVQSIHAWYQRFI', 'MIAHF-YFDYPTTTFRYGKPAWGEETILFGNDMPARWRI-PDHWC-WEYRYVQSIHAWYQRFI']


## Model 3 - flat

In [2]:
H = 64
d= 10
N = 174
n_epochs = 500
nb_PCA_comp=2
loss_type = 'without_J'
family = 'jdoms' #'jdoms_bacteria_train2'
cwd = parent_dir
Q_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_youss/Q_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
K_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_youss/K_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
V_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_youss/V_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
H,d,N=Q_1.shape
q=V_1.shape[1]

# --------------------- PCA 2 models ---------------------
model=AttentionModel(H,d,N,q,Q=Q_1,V=V_1,K=K_1)
torch.sum(model.Q-Q_1)
device = Q_1.device
L = Q_1.shape[-1]
W=attention_heads_from_model(model,Q_1,K_1,V_1)
print(W.shape)

i_indices = torch.arange(L, device=device).unsqueeze(1)
j_indices = torch.arange(L, device=device).unsqueeze(0)
mask = (i_indices != j_indices).float().unsqueeze(0)  # shape (1, L, L)
W = W * mask
    
# Compute Jtens
Jtens = torch.einsum('hri,hab->abri', W, V_1)  # Shape: (q, q, L, L)
q = Jtens.shape[0]
N = Jtens.shape[2]
print(q)
print(N)
print(Jtens.shape)
print(Jtens.shape[-1])

# --------------------- PCA 2 models ---------------------
Q_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_PCA_2models_flat_35_bins/Q_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
K_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_PCA_2models_flat_35_bins/K_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
V_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_PCA_2models_flat_35_bins/V_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
H,d,N1=Q_1.shape
_,_,N2=K_1.shape
_,q1,q2=V_1.shape
model=AttentionModel_PCA(H,d,N1,N2,q1,q2,Q=Q_1,V=V_1,K=K_1)
torch.sum(model.Q-Q_1)
device = Q_1.device
L = Q_1.shape[-1]
W=attention_heads_from_model(model,Q_1,K_1,V_1)
print(W.shape)

# i_indices = torch.arange(L, device=device).unsqueeze(1)
# j_indices = torch.arange(L, device=device).unsqueeze(0)
# mask = (i_indices != j_indices).float().unsqueeze(0)  # shape (1, L, L)
# W = W * mask
    
# Compute Jtens
Jtens_PCA = torch.einsum('hri,hab->abri', W, V_1)  # Shape: (q, q, L, L)
q = Jtens.shape[0]
N = Jtens.shape[2]
print(q)
print(N)
print(Jtens_PCA.shape)

torch.Size([64, 63, 63])
21
63
torch.Size([21, 21, 63, 63])
63
torch.Size([64, 63, 1])
21
63
torch.Size([21, 1225, 63, 1])


In [4]:
def flatten(coords, nb_bins_PCA=35):
    """
    Flatten 2D coordinates into a single index.
    
    Args:
        coords: tuple/list/array of (x, y) coordinates
        nb_bins_PCA: number of bins along one dimension (assumed square grid)
    
    Returns:
        int: flattened coordinate index
    """
    x, y = coords
    return x * nb_bins_PCA + y

In [16]:
"""
    Generate sequences with PLM random initialization
"""
save_dir = "generated_sequences_2Model_PCA_flat"
N_seqs = 30000
save_name = "generated_sequences_randinit_15000"
#generate_plm_n_save(save_dir, save_name, Jtens, N_seqs=N_seqs, init_sequence=None,nb_PCA_comp=2,PCA_comp_list=np.array([20,20]),J_PCA=Jtens_PCA)
N_seqs=15000
#betas = [0.01, 0.5, 1, 2, 10]
betas = [50, 100, 1000]
for b in betas:
    save_name = f"gen_seqs_randinit_Ns{N_seqs}_b{b}_b_PCA_1_c17_7"
    #target_coords = (23, 23)  # as a tuple or list, not numpy array
    target_coords = (17, 7)  # as a tuple or list, not numpy array
    target_coords_flat = flatten(target_coords, nb_bins_PCA=35) 
    #generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=None,nb_PCA_comp=1, beta=b,J_PCA=Jtens_PCA,PCA_comp_list = np.array([target_coords_flat]))
    if b!=1:
        save_name = f"gen_seqs_randinit_Ns{N_seqs}_b_1_b_PCA{b}_c17_7"
        generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=None, beta=1,nb_PCA_comp=1,J_PCA=Jtens_PCA,beta_PCA=b,PCA_comp_list=np.array([target_coords_flat]))


here


100%|██████████| 15000/15000 [05:27<00:00, 45.82it/s]


Generated sequences (letters): ['VHPECDIFCSCLMDQLFRRWCKKPVAICTQKAIYMSDCCRGTQQTYTRVAFK-YKSPRRYYDA', 'VHPECDIFCSCLMDQLFRRWCKKPVAICTQKAIYMSDCCRGTQQTYVRVAFK-YKSPRRYYDA', 'VHPECDIFCSCLMDQ-FRRWCKKPVAICTQKAIYMSDCCRGTQQTYVRVAFK-YKSPRRYYDA', 'VHPECDIFCSCLMDQ-FRRWKKKPVAICTQKAIYMSDCCRGTQQTYVRVAFK-YKSPRRYYDA', 'VHPECDIFCSCLMDQ-FRRWKKKPVAICTQKAIYMSDCCRGTQDTYVRVAFK-YKSPRRYYDA']
Generated sequences saved to generated_sequences_2Model_PCA_flatPCA_comp
here


100%|██████████| 15000/15000 [05:18<00:00, 47.12it/s]


Generated sequences (letters): ['PVKTCAGWRTHRVEFMN-QIIPTKDPM-CDIIEKHEIWHWQVIFDRLLKSHHLMMYLFDCRLT', 'PVKTCAGWRTHRVEFMN-QIIPTKDPM-CDIIEKHEIWHWQVIFDTLLKSHHLMMYLFDCRLT', 'PVKTCAGWRTHRVEFMN-QIIPTKDPM-CDIIEKHEIWHWQVIFDTLLKSHHLMMYLSDCRLT', 'PVKTCAGWRTHRDEFMN-QIIPTKDPM-CDIIEKHEIWHWQVIFDTLLKSHHLMMYLSDCRLT', 'PVKTCAGWRTHRDEFMN-QIIPTKDPM-CDIYEKHEIWHWQVIFDTLLKSHHLMMYLSDCRLT']
Generated sequences saved to generated_sequences_2Model_PCA_flatPCA_comp
here


100%|██████████| 15000/15000 [05:34<00:00, 44.89it/s]


Generated sequences (letters): ['W-RSNVPTQPCFFAIKIICYLGFGPFISRTGNSYMWVHCAKYQVYFMTEWAIMMHQHLLPDPH', 'W-RSNVPTHPCFFAIKIICYLGFGPFISRTGNSYMWVHCAKYQVYFMTEWAIMMHQHLLPDPH', 'W-RSNVPTHPCFFAIKIICYLGFGPFISHTGNSYMWVHCAKYQVYFMTEWAIMMHQHLLPDPH', 'W-RSNVPTHPCFFAIKIICYLGFGPFISHTGNSYMWVHCAKYQVYFMTEWAIMMHQHLLPDPG', 'W-RSNVPTHPCFFAIKIICYLGFGPFISHTGNSYMWVHCAKYQVYFMTEWAIMMHQHLKPDPG']
Generated sequences saved to generated_sequences_2Model_PCA_flatPCA_comp


In [ ]:
save_dir = "generated_sequences_2Model_PCA_multiple_flat"
N_seqs = 10000
# take 10 random coordinates from the PCA space (0-34)
#target_coords_list = [(np.random.randint(0, 35), np.random.randint(0, 35)) for _ in range(10)]
target_coords_list = [(5, 10), (15, 10), (25, 10), (20, 6), (24, 24), (28, 26)]
target_coords_flat = []
for target_coords in target_coords_list:
    target_coords_flat.append(flatten(target_coords, nb_bins_PCA=35))
for target_coords in target_coords_flat:
    save_name = f"gen_seqs_randinit_Ns{N_seqs}_b1_b_PCA_1_c{target_coords}"
    generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=None, beta=1,nb_PCA_comp=1,J_PCA=Jtens_PCA,beta_PCA=1,PCA_comp_list=np.array([target_coords]))



target_coords_flat: [185, 535, 885, 706, 864, 1006]


TypeError: object of type 'int' has no len()